# Knihovna Scikit-learn, přehled užitečných funkcí

V předešlé hodině jsme se vyhýbali programování, jak jen se to dalo. Teď už si ale chceš také sama vše vyzkoušet.
Abys mohla úlohu rozmyšlenou v domácím úkolu naprogramovat, projdeme si nejdůležitější funkce, které budeš potřebovat. 

Především budeme používat knihovnu [Sciki-learn](https://scikit-learn.org) a samozrejmě také pandas. 
Potřebné věci projdeme na příkladu. 

In [1]:
import pandas as pd

## Načtení a příprava dat 

Na začátku vždy bude potřeba připravit data. Čištění dat a použití knihovny pandas už bys měla ovládat, 
zaměříme se jen na věci, které jsou specifické pro strojové učení.

Načíst data tedy umíš.

In [2]:
df_platy = pd.read_csv("static/salaries.csv", index_col=0)
df_platy.sample(10)

,rank,discipline,yrs.since.phd,yrs.service,sex,salary
91,AsstProf,B,10,5,Female,97032
193,Prof,B,19,18,Male,122100
81,Prof,B,43,28,Male,150743
68,Prof,B,35,31,Male,99418
141,AssocProf,A,14,8,Male,100102
70,Prof,B,28,36,Male,91412
121,Prof,A,14,14,Male,115313
143,Prof,A,19,11,Male,106608
48,Prof,B,23,19,Female,151768
86,Prof,B,15,14,Male,132825


Pro predikci použijeme jako příznaky `rank`, `discipline`, `yrs.since.phd`, `yrs.service` a `sex`, 
predikovat budeme hodnotu `salary`.  

Pro učení potřebujeme všechny hondoty převést na čísla (`float`). Pokud by data obsahovala chybějící
hodnoty, nejjednodušší řešení je takové řádky zahodit. (Bonus: pokud bys měla data s větším množstvím
chybějících hodnot, podívej se na možnosti [sklearn.impute](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.impute))

Důležité je vypořádat se s kategorickými hodnotami. Sloupce obsahující hodnoty typu Boolean nebo dvě hodnoty (např. muž/žena), lze snadno převést na hodnoty $[0,1]$. 

In [3]:
df_platy = df_platy.replace({"Male": 0, "Female": 1})
df_platy.sample(10)

,rank,discipline,yrs.since.phd,yrs.service,sex,salary
77,Prof,B,17,3,0,150480
128,AsstProf,A,2,0,1,72500
35,AsstProf,B,4,2,1,80225
131,AssocProf,A,11,9,0,83001
8,Prof,B,45,45,0,147765
180,AsstProf,B,3,3,1,92000
113,AsstProf,A,3,1,0,72500
83,Prof,B,22,20,0,144640
164,AsstProf,B,3,3,0,89942
102,Prof,B,28,23,0,126933


Pro kategorické proměnné s více možnostmi použijeme tzv. *onehot encoding*. 

Např. sloupec `rank` obsahuje hodnoty `Prof`, `AsstProf` a `AssocProf`. K zakódování pomocí onehot encoding potřebujeme tři sloupce: 

Původní hodnota | Kód 
--- | --- 
Prof      | 1 0 0
AsstProf  | 0 1 0 
AssocProf | 0 0 1  


Knihovna Scikitlearn nabízí [sklearn.preprocessing.OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder), při práci s pandas však můžeme použít rovnou metodu [get_dummies](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html). (Pozn. *dummies* proto, že nám přibudou pomocné proměnné (sloupce), které se označují jako *dummy variables*.)

In [4]:
df_platy = pd.get_dummies(df_platy)
df_platy

,yrs.since.phd,yrs.service,sex,salary,rank_AssocProf,rank_AsstProf,rank_Prof,discipline_A,discipline_B
1,19,18,0,139750,0,0,1,0,1
2,20,16,0,173200,0,0,1,0,1
3,4,3,0,79750,0,1,0,0,1
4,45,39,0,115000,0,0,1,0,1
5,40,41,0,141500,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...
194,19,19,0,86250,1,0,0,0,1
195,48,53,0,90000,1,0,0,0,1
196,9,7,0,113600,1,0,0,0,1
197,4,4,0,92700,0,1,0,0,1


Poslední krok předzpracování bývá přeškálování hodnot. Není to vždy nutné, ale některým modelům to může pomoci.
Využijeme [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler).

StandardScaler nám hodnoty přeškáluje, aby zhruba odpovídaly normálnímu rozdělení. Některé algoritmy to předpokládají. Může se pak např. stát, že příznak (sloupeček), která má výrazně větší rozptyl než ostatní, je brán jako významnější. 


(Pozn.: vytvoříme zvlášť objekt na transformaci odezvy. Ten se nám bude později hodit, až budeme chtít z odezvy modelu vidět, jaká je opravdová hodnota platu. Tedy zkonvertovat predikované hodnoty zpět na hodnotu platu.). 

In [5]:
from sklearn.preprocessing import StandardScaler 

priznaky_ke_konverzi = ["yrs.since.phd", "yrs.service"]
odezva = ["salary"]

transformace = StandardScaler()
df_platy[priznaky_ke_konverzi] = transformace.fit_transform(df_platy[priznaky_ke_konverzi])

transformace_odezva = StandardScaler()
df_platy[odezva] = transformace_odezva.fit_transform(df_platy[odezva])

df_platy.sample(10)


,yrs.since.phd,yrs.service,sex,salary,rank_AssocProf,rank_AsstProf,rank_Prof,discipline_A,discipline_B
198,-1.238251,-0.940387,0,-0.669128,0,1,0,0,1
113,-1.319785,-1.192571,0,-1.352272,0,1,0,1,0
143,-0.015236,-0.351955,0,-0.157364,0,0,1,1,0
122,1.044710,1.413339,0,0.462756,0,0,1,1,0
183,-0.912114,-0.604140,0,-0.388863,1,0,0,0,1
186,1.126244,1.245216,0,0.802261,0,0,1,0,1
64,-0.667511,-0.351955,1,-0.262289,1,0,0,0,1
118,1.615450,1.749586,0,0.224741,0,0,1,1,0
162,0.555504,0.320538,0,2.291167,0,0,1,0,1
34,-1.238251,-1.108510,0,-1.081642,0,1,0,0,1


## Vytvoření trénovací a testovací množiny

V teorii strojového učení se vstupy modelu (příznaky, vstupní proměnné) typicky označují písmenem `X` a výstupy písmenem `y`. Řada programátorů toto používá i k označování proměnných v kódu. 
`X` představuje *matici* (neboli tabulku), kde každý řádek odpovídá jednomu datovému vzorku a každý sloupec jednomu příznaku (vstupní proměnné). `y` je vektor, neboli jeden sloupec s odezvou. 

Na vyzobnutí odezvy se výborně hodí metoda [pop](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pop.html).

In [6]:
y = df_platy.pop("salary")
X = df_platy 

print(X.columns)
print(y.name)

Index(['yrs.since.phd', 'yrs.service', 'sex', 'rank_AssocProf',
       'rank_AsstProf', 'rank_Prof', 'discipline_A', 'discipline_B'],
      dtype='object')
salary


Zbývá data rozdělit na trénovací a testovací. K tomu slouží metoda [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html?highlight=train%20test%20split#sklearn.model_selection.train_test_split). 
Data nám rozdělí náhodně na trénovací a testovací sadu. Velikost testovací množiny můžeme specifikovat parametrem `test_size`, jeho defaultní hodnota je `0.25`, t. j. 25%.

In [7]:
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Modely 

Můžeme přejít k samotnému učení. Vybereme si model. Přehled modelů najdeš v sekci [Supervised learnig](https://scikit-learn.org/stable/supervised_learning.html#supervised-learning).
                                                                                                       
                                        
Na regresi můžeš použít:
  - TODO
   -
 
Na klasifikační úlohy (ke kterým se dostaneme v této hodině) využiješ: 
  - TODO
  -

Vytvoříme instanci vybraného modelu (jde nám teď jen o způsob použití knihovny, vezmeme nejjednodušší lineární regresi):

In [8]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

## Trénování

Model natrénujeme na trénovací množině:

In [9]:
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

## Predikce 

Natrénovaný model typicky chceme použít k ohodnocení nějakých nových datových vzorků, k tomu máme metodu `predict`. Zavolejme ji jak na trénovací, tak na testovací data.

In [10]:
train_predikce = model.predict(X_train)
test_predikce = model.predict(X_test)

Pozor, k učení jsme použili transformované hodnoty odezvy. Zajímají-li nás skutečné hodnoty platů, 
musíme i predikované hodnoty transformovat zpět.

In [11]:
# vypišme prvních deset tetovacích vzorků a jejich predikce
odezva = transformace_odezva.inverse_transform(test_predikce)
print(f"odezva modelu    predicke platu")
for i in range(5):
    print(f"{test_predikce[i]:>10.2f}         {odezva[i]:>10.2f}")


odezva modelu    predicke platu
      0.61          128493.18
      0.03          111944.92
      0.13          114720.50
      0.20          116880.33
      0.06          112748.95


## Evaluace modelu

Můžeme využít funkci `score`, která nám vrátí hodnotu $R^2$ metriky:   

In [12]:
print("R2 na trénovací množině: ", model.score(X_train, y_train))
print("R2 na testovací množině: ", model.score(X_test, y_test))

R2 na trénovací množině:  0.5452952238850547
R2 na testovací množině:  0.41408627857916414


Funkce pro všechny možné metriky najdeš v [sklearn.metrics](https://scikit-learn.org/stable/modules/classes.html?highlight=sklearn%20metrics#module-sklearn.metrics).
                                                            (nyní nás zajímají [regresní metriky](https://scikit-learn.org/stable/modules/classes.html?highlight=sklearn%20metrics#module-sklearn.metrics))          

In [13]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

MAE_train = mean_absolute_error(y_train, train_predikce)
MAE_test  = mean_absolute_error(y_test, test_predikce)
MSE_train = mean_squared_error(y_train, train_predikce) 
MSE_test = mean_absolute_error(y_test, test_predikce)
R2_train = r2_score(y_train, train_predikce)
R2_test = r2_score(y_test, test_predikce)

print("Trénovací data  Testovací data")
print(f"MAE {MAE_train:>10.3f} {MAE_test:>10.3f}")
print(f"MAE {MAE_train:>10.3f} {MAE_test:>10.3f}")

Trénovací data  Testovací data
MAE      0.464      0.558
MAE      0.464      0.558


In [14]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

MAE_train = mean_absolute_error(y_train, train_predikce)
MAE_test  = mean_absolute_error(y_test, test_predikce)
MSE_train = mean_squared_error(y_train, train_predikce) 
MSE_test = mean_absolute_error(y_test, test_predikce)
R2_train = r2_score(y_train, train_predikce)
R2_test = r2_score(y_test, test_predikce)

print("     Trénovací data  Testovací data")
print(f"MAE {MAE_train:>15.3f} {MAE_test:>15.3f}")
print(f"MSE {MAE_train:>15.3f} {MAE_test:>15.3f}")
print(f"R2  {MAE_train:>15.3f} {MAE_test:>15.3f}")


     Trénovací data  Testovací data
MAE           0.464           0.558
MSE           0.464           0.558
R2            0.464           0.558


## Uložení modelu 

Někdy si potřebujeme naučený model uchovat na další použití. Model lze uložit do souboru a zase načíst pomocí `pickle`.
Kujme pikle:

In [15]:
import pickle 

with open("model.pickle", "wb") as soubor:
    pickle.dump(model, soubor)


with open("model.pickle", "rb") as soubor:
    staronovy_model = pickle.load(soubor)

staronovy_model.score(X_test, y_test)

0.41408627857916414

Pozn.: Bohužel mám poměrně bohaté špatné zkušenosti z načítáním modelů uložených před delším časem
(bývá problém načíst model uložený ve starší verzi Scikit-learn ve verzi novější). 

### Bonusy:

 - volba vhodného modelu a jeho hyper-parametrů se skrývá pod klíčovým slovem **model selection**. Knihovna Scikit-learn obsahuje různé pomůcky k ulehčení toho výběru. Přesahuje to ale rámec tohoto kurzu, narazíš-li na to toho téma při samostudiu, pročti si [sklear.model_selection](https://scikit-learn.org/stable/modules/classes.html?highlight=model%20selection#module-sklearn.model_selection). 
 
 - v příkladu výše jsme použili různé transformace nad daty a pak teprve tvorbu modelu. Až budeš v těchto věcech zběhlejší, bude se ti hodit propojit tyto věci dohromady. K tomu slouží tzv. [pipeline](https://scikit-learn.org/stable/modules/classes.html?highlight=pipeline#module-sklearn.pipeline).  